In [1]:
'''
Mine
'''
# %tensorflow_version 1.x
# !pip install pyDOE

'\nMine\n'

# Imports

In [2]:
import tensorflow as tf
print(tf.version)
print(tf.test.is_built_with_cuda())
print(tf.test.is_gpu_available(cuda_only=False, min_cuda_compute_capability=None))

<module 'tensorflow._api.v1.version' from 'I:\\Users\\Ragav\\miniconda3\\envs\\juptenflowgpu115\\lib\\site-packages\\tensorflow_core\\_api\\v1\\version\\__init__.py'>
True
True


In [3]:
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
# from plotting import newfig, savefig
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [4]:
import numpy as np
import scipy.io
from scipy.interpolate import griddata
import time
from pyDOE import lhs

In [5]:
%matplotlib widget

In [6]:
lb = np.array([0.0, 0.0])
ub = np.array([10.0, 5.0])

In [7]:
N = 10
fig = plt.figure()
ax = fig.gca()
ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])
ax.set_xticks(np.arange(lb[0],ub[0],(ub[0] - lb[0])/N))
ax.set_yticks(np.arange(lb[1],ub[1],(ub[1] - lb[1])/N))
plt.grid()
l = lb + (ub - lb) * lhs(2, N) 
plt.scatter(l[:, 0], l[:, 1], color="r", label="lhs")
plt.title("Latin Hypercube Sampling\nN=10")
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
fig.set_figheight(3.8)
fig.set_figwidth(6)
plt.tight_layout()


Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

# Load Data

# Model

In [8]:
# Layers
u_layers = [2, 50, 50, 50, 50, 1]
pde_layers = [3, 100, 100, 1]

layers = [2, 100, 100, 100, 100, 50, 1]

# layers = [2, 50, 50, 50, 50, 50, 1]

In [9]:
# tf placeholders for Identification
t_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_tf = tf.placeholder(tf.float32, shape=[None, 1])
v_tf = tf.placeholder(tf.float32, shape=[None, 1])
t_tf, x_tf, u_tf, v_tf

(<tf.Tensor 'Placeholder:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_1:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_2:0' shape=(?, 1) dtype=float32>,
 <tf.Tensor 'Placeholder_3:0' shape=(?, 1) dtype=float32>)

In [10]:
def initialize_NN(layers):
    weights = []
    biases = []
    num_layers = len(layers)
    for l in range(0, num_layers - 1):
        W = xavier_init(size=[layers[l], layers[l + 1]])
        b = tf.Variable(tf.zeros([1, layers[l + 1]], dtype=tf.float32),
                        dtype=tf.float32)
        weights.append(W)
        biases.append(b)
    return weights, biases

def xavier_init(size):
    in_dim = size[0]
    out_dim = size[1]
    xavier_stddev = np.sqrt(2 / (in_dim + out_dim))
    return tf.Variable(tf.truncated_normal([in_dim, out_dim],
                                           stddev=xavier_stddev,
                                           dtype=tf.float32),
                       dtype=tf.float32)

In [11]:
def neural_net(X, weights, biases):
    num_layers = len(weights) + 1
    H = X
    for l in range(0, num_layers - 2):
        W = weights[l]
        b = biases[l]
        H = tf.sin(tf.add(tf.matmul(H, W), b))
    W = weights[-1]
    b = biases[-1]
    Y = tf.add(tf.matmul(H, W), b)
    return Y

In [12]:
u_weights, u_biases = initialize_NN(layers)
v_weights, v_biases = initialize_NN(layers)

In [13]:
lb_tf = tf.placeholder(tf.float32, shape=[2])
ub_tf = tf.placeholder(tf.float32, shape=[2])

In [14]:
# tf placeholders for Solution
t0_tf = tf.placeholder(tf.float32, shape=[None, 1])
x0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u0_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_t0_tf = tf.placeholder(tf.float32, shape=[None, 1])
v0_tf = tf.placeholder(tf.float32, shape=[None, 1])
v_t0_tf = tf.placeholder(tf.float32, shape=[None, 1])

t_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
v_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
u_x_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
v_x_lb_tf = tf.placeholder(tf.float32, shape=[None, 1])
# t_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
# x_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
# u_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])
# v_ub_tf = tf.placeholder(tf.float32, shape=[None, 1])

t_f_tf = tf.placeholder(tf.float32, shape=[None, 1])
x_f_tf = tf.placeholder(tf.float32, shape=[None, 1])

$$ y = x $$

In [15]:
m = 2

In [16]:
def sol_net_u(t, x):
    X = tf.concat([t, x], 1)
    H = 2.0 * (X - lb_tf) / (ub_tf - lb_tf) - 1.0
    u = neural_net(H, u_weights, u_biases)
    v = neural_net(H, v_weights, v_biases)
    u_x = tf.gradients(u, x)[0]
    v_x = tf.gradients(v, x)[0]
    u_t = tf.gradients(u, t)[0]
    v_t = tf.gradients(v, t)[0]
    return u, v, u_t, v_t, u_x, v_x

def sol_net_f(t, x):
    u, v, u_t, v_t, u_x, v_x = sol_net_u(t, x)

    u_xx = tf.gradients(u_x, x)[0]
    v_xx = tf.gradients(v_x, x)[0]

    # terms = tf.concat([u, u_x, u_xx], 1)

    # f = u_t - net_pde(terms)
    f = - v_t + u_xx / (2 * m)
    g = u_t + v_xx / (2 * m)

    return f, g

In [17]:
# tf graphs for Solution
u0_pred, v0_pred, u_t0_pred, v_t0_pred, u_x0_pred, v_x0_pred = sol_net_u(t0_tf, x0_tf)
u_lb_pred, v_lb_pred, u_t_lb_pred, v_t_lb_pred, u_x_lb_pred, v_x_lb_pred = sol_net_u(t_lb_tf, x_lb_tf)
# u_ub_pred, v_ub_pred, u_x_ub_pred, v_x_ub_pred = sol_net_u(t_ub_tf, x_ub_tf)
sol_f_pred, sol_g_pred = sol_net_f(t_f_tf, x_f_tf)

In [18]:
# loss for Solution
sol_loss =  tf.reduce_sum(tf.square(u0_tf - u0_pred)) + \
            tf.reduce_sum(tf.square(v0_tf - v0_pred)) + \
            tf.reduce_sum(tf.square(u_t0_tf - u_t0_pred)) + \
            tf.reduce_sum(tf.square(v_t0_tf - v_t0_pred)) + \
            tf.reduce_sum(tf.square(u_lb_tf - u_lb_pred)) + \
            tf.reduce_sum(tf.square(v_lb_tf - v_lb_pred)) + \
            tf.reduce_sum(tf.square(sol_f_pred)) + \
            tf.reduce_sum(tf.square(sol_g_pred))
            # tf.reduce_sum(tf.square(u_x_lb_tf - u_x_lb_pred)) + \
            # tf.reduce_sum(tf.square(v_x_lb_tf - v_x_lb_pred)) + \

In [19]:
# Optimizer for Solution
sol_optimizer = tf.contrib.opt.ScipyOptimizerInterface(
    sol_loss,
    var_list = u_weights + u_biases + v_weights + v_biases,
    method='L-BFGS-B',
    options={
        'maxiter': 50000,
        'maxfun': 50000,
        'maxcor': 50,
        'maxls': 50,
        'ftol': 1.0 * np.finfo(float).eps
    })

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.



In [20]:
adam_optimizer = tf.train.AdamOptimizer()
sol_train_op_Adam = adam_optimizer.minimize(
            sol_loss,
            var_list= u_weights + u_biases + v_weights + v_biases,)

In [21]:
# tf session
sess = tf.Session(config=tf.ConfigProto(
    allow_soft_placement=True, log_device_placement=True))
init = tf.global_variables_initializer()
sess.run(init)

Device mapping:
/job:localhost/replica:0/task:0/device:GPU:0 -> device: 0, name: NVIDIA GeForce GTX 1650 Ti, pci bus id: 0000:01:00.0, compute capability: 7.5



# Training

## Prepare data

In [22]:
lb = np.array([0.0, 0.0])
ub = np.array([10.0, 5.0])

In [23]:
N0 = 200
N_b = 400
N_f = 20000
(N0, N_b, N_f)

(200, 400, 20000)

In [24]:
t_data = np.linspace(lb[0], ub[0], N_b)[:, None]
x_data = np.linspace(lb[1], ub[1], N0)[:, None]

In [25]:
T_sol, X_sol = np.meshgrid(t_data, x_data)
# U_sol = u_data

In [26]:
X_sol_star = np.hstack(
    (T_sol.flatten()[:, None],
    X_sol.flatten()[:, None])
    )
# U_sol_star = U_sol.flatten()[:, None]

print(X_sol_star.shape, X_sol_star[0:10], sep='\n')

(80000, 2)
[[0.         0.        ]
 [0.02506266 0.        ]
 [0.05012531 0.        ]
 [0.07518797 0.        ]
 [0.10025063 0.        ]
 [0.12531328 0.        ]
 [0.15037594 0.        ]
 [0.1754386  0.        ]
 [0.20050125 0.        ]
 [0.22556391 0.        ]]


In [27]:
A = 1
p = 1.5
m = 2
E = p**2 / (2 * m)

In [28]:
u_data = A * np.cos(p * x_data)
v_data = - A * np.sin(p * x_data)

u_t_data = A * E * np.sin(p * x_data)
v_t_data = A * E * np.cos(p * x_data)

In [29]:
fig, ax = plt.subplots()
ax.set_xlabel('$x$')
ax.set_ylabel('$u$')
ax.plot(x_data[:, 0], u_data[:, 0:1])
ax.plot(x_data[:, 0], v_data[:, 0:1])
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [30]:
fig, ax = plt.subplots()
ax.set_xlabel('$x$')
ax.set_ylabel('$u$')
ax.plot(x_data[:, 0], u_t_data[:, 0:1])
ax.plot(x_data[:, 0], v_t_data[:, 0:1])
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [31]:
X0 = np.hstack((T_sol[:, 0:1], X_sol[:, 0:1])) # left boundary
X_lb = np.hstack((T_sol[0:1, :].T, X_sol[0:1, :].T)) # lower boundary
X_ub = np.hstack((T_sol[0:1, :].T, np.repeat(ub[1], t_data.shape[0])[:, None])) # upper boundary

In [32]:
# shuffled initial boundary data (left boundary)
idx_x = np.random.choice(x_data.shape[0], N0, replace=False)
X0_train = X0[idx_x, :]
u0_train = u_data[idx_x, 0:1]
v0_train = v_data[idx_x, 0:1]
u_t0_train = u_t_data[idx_x, 0:1]
v_t0_train = v_t_data[idx_x, 0:1]

In [33]:
u_lb = A * np.cos(E * t_data)
v_lb = A * np.sin(E * t_data)

In [34]:
fig, ax = plt.subplots()
ax.set_xlabel('$t$')
ax.set_ylabel('$u$')
ax.plot(t_data[:, 0], u_lb[:, 0:1])
ax.plot(t_data[:, 0], v_lb[:, 0:1])
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [35]:
# shuffle time data
idx_t = np.random.choice(t_data.shape[0], N_b, replace=False)
tb_train = t_data[idx_t, :]
ub_train = u_lb[idx_t, :]
vb_train = v_lb[idx_t, :]

In [36]:
X_f_train = lb + (ub - lb) * lhs(2, N_f) 

In [37]:
fig, ax = plt.subplots()
ax.set_xlim(lb[0] -0.1, ub[0])
ax.set_ylim(lb[1] - 0.4, ub[1] + 0.4)
fig.set_figheight(3.2)
fig.set_figwidth(6)

ax.scatter(X0_train[:, 0], X0_train[:, 1], s=4, marker='.')
ax.scatter(tb_train[:, 0], np.repeat(lb[1], N_b), s=4, marker='.')
ax.scatter(tb_train[:, 0], np.repeat(ub[1], N_b), s=4, marker='.')
ax.scatter(X_f_train[:, 0], X_f_train[:, 1], s=4, marker='.', edgecolors='none')
# ax.imshow(u0_train, extent=(t_data[0, 0], t_data[1, 0], x0_train.max(), x0_train.min()), aspect='auto')
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [38]:
def callback(loss):
    print('Loss: %e' % (loss))

In [39]:
tf_dict = {
    lb_tf: lb,
    ub_tf: ub,
    t0_tf: X0_train[:, 0:1],
    x0_tf: X0_train[:, 1:2],
    u0_tf: u0_train,
    v0_tf: v0_train,
    u_t0_tf: u_t0_train,
    v_t0_tf: v_t0_train,
    t_lb_tf: tb_train,
    x_lb_tf: X_lb[:, 1:2],
    # t_ub_tf: X_ub[:, 0:1],
    # x_ub_tf: X_ub[:, 1:2],
    u_lb_tf: ub_train,
    v_lb_tf: vb_train,
    t_f_tf: X_f_train[:, 0:1],
    x_f_tf: X_f_train[:, 1:2]
}

In [40]:
start_time = time.time()
it = 0
end = False
while not(end):

    sess.run(sol_train_op_Adam, tf_dict)

    # Print
    if it % 10 == 0:
        elapsed = time.time() - start_time
        loss_value = sess.run(sol_loss, tf_dict)
        print('It: %d, Loss: %.3e, Time: %.2f' %
                (it, loss_value, elapsed))
        start_time = time.time()

        if loss_value < 50 * 10**(-1):
            end = True

    it = it + 1

It: 0, Loss: 7.228e+02, Time: 2.52
It: 10, Loss: 6.436e+02, Time: 1.05
It: 20, Loss: 6.332e+02, Time: 1.05
It: 30, Loss: 6.320e+02, Time: 1.05
It: 40, Loss: 6.291e+02, Time: 1.05
It: 50, Loss: 6.083e+02, Time: 1.05
It: 60, Loss: 5.903e+02, Time: 1.05
It: 70, Loss: 5.673e+02, Time: 1.05
It: 80, Loss: 5.284e+02, Time: 1.05
It: 90, Loss: 4.921e+02, Time: 1.05
It: 100, Loss: 4.494e+02, Time: 1.05
It: 110, Loss: 4.061e+02, Time: 1.05
It: 120, Loss: 3.733e+02, Time: 1.05
It: 130, Loss: 3.272e+02, Time: 1.05
It: 140, Loss: 2.955e+02, Time: 1.05
It: 150, Loss: 2.846e+02, Time: 1.05
It: 160, Loss: 2.504e+02, Time: 1.05
It: 170, Loss: 2.351e+02, Time: 1.05
It: 180, Loss: 2.231e+02, Time: 1.05
It: 190, Loss: 2.166e+02, Time: 1.05
It: 200, Loss: 2.141e+02, Time: 1.05
It: 210, Loss: 2.068e+02, Time: 1.05
It: 220, Loss: 1.996e+02, Time: 1.05
It: 230, Loss: 1.945e+02, Time: 1.05
It: 240, Loss: 2.004e+02, Time: 1.05
It: 250, Loss: 1.850e+02, Time: 1.05
It: 260, Loss: 1.774e+02, Time: 1.05
It: 270, Los

KeyboardInterrupt: 

In [41]:
sol_optimizer.minimize(sess,
                       feed_dict=tf_dict,
                       fetches=[sol_loss],
                       loss_callback=callback)

Loss: 5.980099e+01
Loss: 1.090219e+05
Loss: 5.842398e+01
Loss: 5.596005e+01
Loss: 5.440953e+01
Loss: 5.181392e+01
Loss: 5.196633e+01
Loss: 5.149715e+01
Loss: 5.141519e+01
Loss: 5.127087e+01
Loss: 5.113522e+01
Loss: 5.098820e+01
Loss: 5.080380e+01
Loss: 5.063353e+01
Loss: 5.053558e+01
Loss: 5.042136e+01
Loss: 5.033751e+01
Loss: 5.021410e+01
Loss: 5.013660e+01
Loss: 5.008208e+01
Loss: 5.000476e+01
Loss: 4.994364e+01
Loss: 4.981167e+01
Loss: 4.973695e+01
Loss: 4.967474e+01
Loss: 4.960762e+01
Loss: 4.951656e+01
Loss: 4.934472e+01
Loss: 4.927836e+01
Loss: 4.916519e+01
Loss: 4.911151e+01
Loss: 4.904112e+01
Loss: 4.888181e+01
Loss: 4.863039e+01
Loss: 4.863512e+01
Loss: 4.848633e+01
Loss: 4.830696e+01
Loss: 4.815942e+01
Loss: 4.795971e+01
Loss: 4.748894e+01
Loss: 4.701089e+01
Loss: 4.665791e+01
Loss: 4.649481e+01
Loss: 4.635198e+01
Loss: 4.608460e+01
Loss: 4.576703e+01
Loss: 4.544905e+01
Loss: 4.514402e+01
Loss: 4.496610e+01
Loss: 4.480847e+01
Loss: 4.453801e+01
Loss: 4.430365e+01
Loss: 4.4063

In [42]:
sess.run(sol_loss, feed_dict=tf_dict)

3.4004774

In [ ]:
X_sol_star[:, 0:1]

array([[ 0.        ],
       [ 0.02506266],
       [ 0.05012531],
       ...,
       [ 9.94987469],
       [ 9.97493734],
       [10.        ]])

In [ ]:
u_pred = sess.run(u0_pred, {
        lb_tf: lb,
        ub_tf: ub,
        t0_tf: X_sol_star[:, 0:1],
        x0_tf: X_sol_star[:, 1:2]
    })

In [ ]:
fig, ax = plt.subplots()
ax.set_xlabel('$t$')
ax.set_ylabel('$x$')
ax.set_xlim(lb[0], ub[0])
ax.set_ylim(lb[1], ub[1])
fig.set_figheight(3.2)
fig.set_figwidth(6)
plt.pcolormesh(np.reshape(X_sol_star[:, 0], (N0, -1)), 
               np.reshape(X_sol_star[:, 1], (N0, -1)), 
               np.reshape(u_pred[:, 0], (N0, -1)), 
               shading='gouraud',  cmap='jet')
plt.colorbar()
plt.tight_layout()

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [ ]:
plt.show()